In [51]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle


In [52]:
##load the trained model,scaler and the label encoder
model = load_model('model.h5')


# Load the scaler and label encoder
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder = pickle.load(file) 

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

In [53]:
#example input data
input_data = {
    'Age': 40,    
    'CreditScore': 600,
    'Tenure': 3,
    'Balance': 60000,  
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000, 
    'Geography': 'France',
    'Gender':'Male'  
}

In [54]:
#one hot encode the geography
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Projects\ANN Classification\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [55]:
input_df=pd.DataFrame([input_data])
input_df

,Age,CreditScore,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography,Gender
0,40,600,3,60000,2,1,1,50000,France,Male


In [56]:
#encode categorical variables
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,Age,CreditScore,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography,Gender
0,40,600,3,60000,2,1,1,50000,France,1


In [57]:
#combine one hot encoded geography with the input data
input_df=pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df], axis=1)
input_df


,Age,CreditScore,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography_France,Geography_Germany,Geography_Spain
0,40,600,3,60000,2,1,1,50000,1,1.0,0.0,0.0


In [58]:
# Ensure input_df columns are in the same order as during scaler fitting
input_df_ordered = input_df[scaler.feature_names_in_]
# Scale the input data
input_scaled = scaler.transform(input_df_ordered)
input_scaled


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [59]:
#perdict churn
predictions = model.predict(input_scaled)
predictions

1/1 [==============================] - 0s 94ms/step


array([[0.03822409]], dtype=float32)

In [60]:
predictions_proba = predictions[0][0]
predictions_proba

0.03822409

In [61]:
if predictions_proba > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(predictions_proba * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - predictions_proba) * 100))

The customer is unlikely to churn with a probability of 96.18%
